In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score

pd.set_option('display.max_columns', None)

### Loading Data

In [3]:
df = pd.DataFrame()
data_dir = "CICIoT2023"

for file in os.listdir(data_dir):
    f = os.path.join(data_dir, file)
    curr_df = pd.read_csv(f)
    df = pd.concat([df, curr_df])
    break


In [4]:
df.head(3)

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,psh_flag_number,ack_flag_number,ece_flag_number,cwr_flag_number,ack_count,syn_count,fin_count,urg_count,rst_count,HTTP,HTTPS,DNS,Telnet,SMTP,SSH,IRC,TCP,UDP,DHCP,ARP,ICMP,IPv,LLC,Tot sum,Min,Max,AVG,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,0.0,54.00,6.00,64.0,0.329807,0.329807,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,567.00,54.0,54.0,54.000000,0.000000,54.00,8.334383e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,DDoS-RSTFINFlood
1,0.0,57.04,6.33,64.0,4.290556,4.290556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,581.33,54.0,66.3,54.796404,2.822973,57.04,8.292607e+07,9.5,10.464666,4.010353,160.987842,0.05,141.55,DoS-TCP_Flood
2,0.0,0.00,1.00,64.0,33.396799,33.396799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,441.00,42.0,42.0,42.000000,0.000000,42.00,8.312799e+07,9.5,9.165151,0.000000,0.000000,0.00,141.55,DDoS-ICMP_Flood


In [5]:
df.shape

(238687, 47)

In [6]:
df["label"].value_counts()

label
DDoS-ICMP_Flood            36554
DDoS-UDP_Flood             27626
DDoS-TCP_Flood             23149
DDoS-PSHACK_Flood          21210
DDoS-SYN_Flood             20739
DDoS-RSTFINFlood           20669
DDoS-SynonymousIP_Flood    18189
DoS-UDP_Flood              16957
DoS-TCP_Flood              13630
DoS-SYN_Flood              10275
BenignTraffic               5600
Mirai-greeth_flood          5016
Mirai-udpplain              4661
Mirai-greip_flood           3758
DDoS-ICMP_Fragmentation     2377
MITM-ArpSpoofing            1614
DDoS-ACK_Fragmentation      1505
DDoS-UDP_Fragmentation      1484
DNS_Spoofing                 925
Recon-HostDiscovery          697
Recon-OSScan                 517
Recon-PortScan               430
DoS-HTTP_Flood               414
VulnerabilityScan            210
DDoS-HTTP_Flood              169
DDoS-SlowLoris               106
DictionaryBruteForce          63
SqlInjection                  31
BrowserHijacking              30
CommandInjection              28
Back

In [7]:
X = df.loc[:, df.columns != "label"]
y = df["label"]

standard_scaler = StandardScaler()
X = standard_scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"X_train shape = {X_train.shape}")
print(f"y_train shape = {y_train.shape}")
print(f"X_test shape = {X_test.shape}")
print(f"y_test shape = {y_test.shape}")
X[0]


X_train shape = (190949, 46)
y_train shape = (190949,)
X_test shape = (47738, 46)
y_test shape = (47738,)


array([-1.80253163e-02, -1.67424666e-01, -3.42900462e-01, -1.67211843e-01,
       -9.31117565e-02, -9.31117565e-02, -3.05100533e-03,  3.24906971e+00,
       -5.09563604e-01,  3.16732340e+00, -3.12083895e-01, -3.77523675e-01,
        0.00000000e+00,  0.00000000e+00,  3.17441373e+00, -4.97104570e-01,
        2.85180395e+00, -8.67756282e-02, -1.19045354e-01, -2.27289750e-01,
       -2.41396859e-01, -1.29464890e-02,  0.00000000e+00,  0.00000000e+00,
       -6.78878363e-03,  0.00000000e+00,  8.60678187e-01, -5.19352017e-01,
        0.00000000e+00, -8.92235921e-03, -4.41249797e-01,  1.19359230e-02,
        1.19359230e-02, -2.84363641e-01, -2.69827118e-01, -2.41941633e-01,
       -2.93314868e-01, -2.05884713e-01, -2.91609711e-01,  1.12000430e-02,
        3.29118194e-03, -3.17348305e-01, -2.05716259e-01, -7.14054803e-02,
       -4.14431178e-01,  3.29712418e-03])

### Models

In [8]:
ml_models = [
    LogisticRegression(random_state=42, n_jobs=-1),
    RandomForestClassifier(random_state=42)
]

for model in ml_models:
    model.fit(X_train, y_train)
    print(f"finished training {model}")

finished training LogisticRegression(n_jobs=-1, random_state=42)
finished training RandomForestClassifier(random_state=42)


In [9]:
for model in ml_models:
    print(f"{model} accuracy = {model.score(X_test, y_test)}")

LogisticRegression(n_jobs=-1, random_state=42) accuracy = 0.8030499811470946
RandomForestClassifier(random_state=42) accuracy = 0.9916418785872889
